In [ ]:
import psycopg2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import tensorflow as tf
import datetime

In [ ]:
import json
import requests

In [ ]:
window_size = 20

In [ ]:
conn = psycopg2.connect(
    host='localhost',
    database='db',
    user='postgres',
    password='postgres'
)

In [ ]:
cur = conn.cursor()

In [ ]:
cur.execute("SELECT version()")

In [ ]:
db_version = cur.fetchone()

In [ ]:
print(db_version)

In [ ]:
# query all data of the specific stock
para_p_sql = """
    SELECT h.date, h.close, h.high, h.low, h.open, h.capacity, h.turnover, h.transactions, h.stock_code
    FROM history as h
    WHERE h.stock_code = %(stock_code)s;
"""

df = pd.read_sql(para_p_sql, con=conn, params={'stock_code': '0050'})

In [ ]:
para_p_sql = """
    SELECT * FROM (
        SELECT h.date, h.close, h.high, h.low, h.open, h.capacity, h.turnover, h.transactions, h.stock_code
        FROM history as h
        WHERE h.stock_code = %(stock_code)s and h.date <= %(date)s
        ORDER BY h.date DESC
        LIMIT %(window_size)s
    ) AS TEMP
    ORDER BY TEMP.date
"""
df = pd.read_sql(para_p_sql, con=conn, params={'stock_code': '0050', 'date': '2020-05-05', 'window_size': window_size})

In [ ]:
today = datetime.datetime.today().strftime("%Y-%m-%d")
print(today)
df

In [ ]:
matrix = df[['close']].values
time = df['date'].values

In [ ]:
matrix[:9]

In [ ]:
matrix[:window_size].tolist()

In [ ]:
def rest_request():
    url = "http://localhost:8501/v1/models/stocknet:predict"
#     payload = json.dumps({"instances": [matrix[:window_size].tolist(), 
#                                         matrix[1:window_size+1].tolist()]})
    payload = json.dumps({"instances": [matrix[:window_size].tolist()]})
    headers = {"content-type": "application/json"}
    response = requests.post(url, data=payload, headers=headers)
    print(response)
    return response

In [ ]:
res = rest_request()
predictions = json.loads(res.text)['predictions']
print(predictions)

In [ ]:
x = np.array(predictions)
x.shape

In [ ]:
for x in predictions:
    print(x)

In [ ]:
res.text

In [ ]:
all_stock_code_sql = """
    SELECT stock.stock_code
    FROM stock
    ORDER BY stock.stock_code;
"""

In [ ]:
cur.execute(all_stock_code_sql)

In [ ]:
res = cur.fetchall()

In [ ]:
print(res)
stock_codes = [x[0] for x in res]
print(stock_codes)

In [ ]:
select_history_sql = """
    SELECT history.date, history.close, history.stock_code
    FROM history
    ORDER BY history.stock_code, history.date;
"""
# cur.execute(select_history_sql)
# res = cur.fetchall()
# print(res)

In [ ]:
# for x in res[0]:
#     print(x, type(x))

In [ ]:
# for x in res:
#     print(x)

In [ ]:
def plot_series(time, series, start=0, end=None, format="-"):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time")
    plt.ylabel("Value")
    plt.grid(True)

In [ ]:
# p_sql = """
#     SELECT * FROM get_stock_history('%s');
# """ % ("0050")
# df = pd.read_sql(p_sql, con=conn)

In [ ]:
# prepared_sql = """
#     PREPARE get_history AS
#     SELECT h.date, h.close, h.stock_code
#     FROM history as h
#     WHERE h.stock_code = $1;
# """
# cur.execute(prepared_sql)

In [ ]:
para_p_sql = """
    SELECT h.date, h.close, h.high, h.low, h.open, h.capacity, h.turnover, h.transactions, h.stock_code
    FROM history as h
    WHERE h.stock_code = %(stock_code)s;
"""

In [ ]:
print(para_p_sql)

In [ ]:
# cur.execute(p_sql)
# res = cur.fetchall()
# print(res)

In [ ]:
# df = pd.read_sql(para_p_sql, con=conn, params={'stock_code': '006208'})
df = pd.read_sql(para_p_sql, con=conn, params={'stock_code': '0050'})

In [ ]:
df

In [ ]:
# mae: 2.2646
# matrix = df[['close', 'high', 'low', 'open', 'capacity', 'turnover', 'transactions']].values
matrix = df[['close']].values
time = df['date'].values

In [ ]:
print(matrix.shape)

In [ ]:
matrix[:9, :]

In [ ]:
plot_series(time, matrix[:, 0])

In [ ]:
window_size = 20
batch_size = 32
split_time = 500
shuffle_buffer = 1000
dim = matrix.shape[1]

def windowed_dataset_m(matrix, window_size=20, batch_size=32, shuffle_buffer=1000):
#     matrix = tf.expand_dims(matrix, axis=-1)
    dataset = tf.data.Dataset.from_tensor_slices(matrix)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], [window[-1:][0][0]]))
    dataset = dataset.batch(batch_size).prefetch(1)
    return dataset

def model_predict_m(model, matrix, window_size=20):
    ds = tf.data.Dataset.from_tensor_slices(matrix)
    ds = ds.window(window_size, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_size))
    ds = ds.batch(32).prefetch(1)
    forecast = model.predict(ds)
    return forecast

In [ ]:
def raw_model(window_size, dim):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv1D(filters=64,
                               kernel_size=5,
                               strides=1,
                               padding="causal",
                               activation="relu",
                               input_shape=[window_size, dim]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences=True)),
#         tf.keras.layers.LSTM(64, return_sequences=True),
        tf.keras.layers.Dense(30, activation="relu"),
        tf.keras.layers.Dense(10, activation="relu"),
        tf.keras.layers.Dense(1)
    ])
    return model

# Prepare Dataset

In [ ]:
time_train = time[:split_time]
x_train = matrix[:split_time]
time_valid = time[split_time:]
x_val = matrix[split_time:]

train_ds = windowed_dataset_m(x_train, window_size, batch_size, shuffle_buffer)

In [ ]:
# for window in train_ds:
# #     for val in window:
# #         print(val.numpy())
#     x, y = window
#     print(x.shape)
#     print(x.numpy())
#     print('-')
#     print(y.numpy())

#     break

# Experiment on Learning Rate

In [ ]:
model = raw_model(window_size, dim)

lr_schedule = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: 1e-8 * 10**(epoch / 20))

# optimizer = tf.keras.optimizers.SGD(lr=1e-8, momentum=0.9)
optimizer = tf.keras.optimizers.Adam(lr=1e-8)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
model.summary()
history = model.fit(train_ds, epochs=200, callbacks=[lr_schedule])

In [ ]:
plt.semilogx(history.history["lr"], history.history["loss"])
plt.axis([1e-8, 10, 0, 60])

# Train the model

In [ ]:
model = raw_model(window_size, dim)

# optimizer = tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9)
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(loss=tf.keras.losses.Huber(),
              optimizer=optimizer,
              metrics=["mae"])
model.summary()
history = model.fit(train_ds, epochs=300)

# Forecast

In [ ]:
forecast = model_predict_m(model, matrix, window_size)
forecast = forecast[split_time - window_size:-1, -1, 0]

In [ ]:
print(x_val)
print(x_val[:, 0])

In [ ]:
plt.figure(figsize=(10, 6))
plot_series(time_valid, x_val[:, 0])
plot_series(time_valid, forecast)

# Save Model

In [ ]:
import os

In [ ]:
curr_dir = os.getcwd()
print(curr_dir)

In [ ]:
model_save_path = os.path.join(curr_dir, "stocknet/1/")
tf.saved_model.save(model, model_save_path)